# pyannote/diarization-3.0

In [ ]:
!pip install -q torchaudio pyannote.audio huggingface_hub onnxruntime-gpu
!apt-get install -y libsndfile1 ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:

In [ ]:
import os
import torch
from pyannote.audio import Pipeline
from huggingface_hub import notebook_login

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from pyannote.audio import Pipeline

In [ ]:
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.0", use_auth_token=True)

config.yaml:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

speaker-embedding.onnx:   0%|          | 0.00/26.5M [00:00<?, ?B/s]

In [ ]:
audio_files = {
    "no_noise": "/content/no_noise.m4a",
    "some_noise": "/content/some_noise.m4a",
    "heavy_noise": "/content/heavy_noise.m4a"
}

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment

def convert_m4a_to_wav(m4a_path, wav_path):
    audio = AudioSegment.from_file(m4a_path, format="m4a")
    audio.export(wav_path, format="wav")

# Convert all M4A files to WAV
for label, file_path in audio_files.items():
    wav_path = file_path.replace('.m4a', '.wav')
    convert_m4a_to_wav(file_path, wav_path)
    audio_files[label] = wav_path  # Update the file path to the new WAV file

In [ ]:
results = []

for label, file_path in audio_files.items():
    # Perform diarization
    diarization = diarization_pipeline(file_path)

    # Extract speech segments
    speech_segments = [(segment.start, segment.end)
                      for segment, _, speaker in diarization.itertracks(yield_label=True)]

    # Calculate metrics
    audio_duration = diarization.get_timeline().duration()
    speech_duration = sum(end - start for start, end in speech_segments)
    speech_percentage = (speech_duration / audio_duration) * 100

    results.append({
        "file": label,
        "total_duration": f"{audio_duration:.2f}s",
        "speech_duration": f"{speech_duration:.2f}s",
        "speech_percentage": f"{speech_percentage:.2f}%",
        "segments": speech_segments
    })

In [ ]:
print("Voice Activity Detection Results:")
for result in results:
    print(f"\nFile: {result['file']}")
    print(f"Total Duration: {result['total_duration']}")
    print(f"Speech Duration: {result['speech_duration']}")
    print(f"Speech Percentage: {result['speech_percentage']}")
    print(f"Number of Speech Segments: {len(result['segments'])}")

Voice Activity Detection Results:

File: no_noise
Total Duration: 60.45s
Speech Duration: 60.45s
Speech Percentage: 100.00%
Number of Speech Segments: 4

File: some_noise
Total Duration: 51.18s
Speech Duration: 51.18s
Speech Percentage: 100.00%
Number of Speech Segments: 3

File: heavy_noise
Total Duration: 56.09s
Speech Duration: 56.09s
Speech Percentage: 100.00%
Number of Speech Segments: 1


# pyannote/segmentation-3.0

In [ ]:
# Clean environment first
!pip uninstall -y pyannote.audio pyannote.core numpy torch torchaudio
!pip cache purge

# Install with version pinning (validated with CUDA 12.4)
!pip install -q numpy==1.26.4
!pip install -q pyannote.audio==3.3.1 \
                pyannote.core==5.0.0 \
                torch==2.1.0+cu121 \
                torchaudio==2.1.0+cu121 \
                --extra-index-url https://download.pytorch.org/whl/cu121 \
                --force-reinstall

# Fix remaining warnings
import warnings
warnings.filterwarnings("ignore",
    message=".*'get_dtype'.*",
    category=UserWarning)

# Verify core imports
from pyannote.audio import Model
from pyannote.audio.pipelines import VoiceActivityDetection

In [ ]:
import torch
from pyannote.audio import Pipeline
import librosa
import numpy as np
import subprocess
from huggingface_hub import notebook_login
from getpass import getpass

In [ ]:
print("Enter your Hugging Face access token:")
auth_token = getpass()
notebook_login(auth_token)

In [ ]:
def convert_to_wav(input_file):
    output_file = input_file.replace('.m4a', '.wav')
    command = f"ffmpeg -i {input_file} -ac 1 -ar 16000 {output_file} -y"
    subprocess.call(command, shell=True)
    return output_file

In [ ]:
model = Model.from_pretrained("pyannote/segmentation-3.0", use_auth_token=auth_token)
pipeline = VoiceActivityDetection(segmentation=model)
HYPER_PARAMETERS = {
    "min_duration_on": 0.1,  # Remove speech < 0.1s
    "min_duration_off": 0.1   # Fill silence < 0.1s
}

pipeline = VoiceActivityDetection(segmentation=model)
pipeline.instantiate(HYPER_PARAMETERS)

In [ ]:
def analyze_audio(file_path):
    # Convert to WAV
    wav_path = convert_to_wav(file_path)

    # Load audio
    waveform, sample_rate = librosa.load(wav_path, sr=16000, mono=True)

    # Process with VAD
    input = {"waveform": torch.from_numpy(waveform[None, :]), "sample_rate": sample_rate}
    output = pipeline(input)

    # Calculate speech percentage
    total_duration = len(waveform) / sample_rate
    speech_duration = sum([segment.duration for segment in output.get_timeline().support()])
    speech_percentage = (speech_duration / total_duration) * 100

    return {
        "file": file_path,
        "total_duration": total_duration,
        "speech_segments": list(output.get_timeline().support()),
        "speech_duration": speech_duration,
        "speech_percentage": speech_percentage,
        "sample_rate": sample_rate
    }

In [ ]:
files = ["/content/no_noise.m4a", "/content/some_noise.m4a", "/content/heavy_noise.m4a"]
results = []

for file in files:
    try:
        results.append(analyze_audio(file))
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")

print("\nVAD Results:")
for result in results:
    print(f"\nFile: {result['file']}")
    print(f"Total duration: {result['total_duration']:.2f}s")
    print(f"Speech duration: {result['speech_duration']:.2f}s")
    print(f"Speech percentage: {result['speech_percentage']:.2f}%")
    print(f"Sample rate: {result['sample_rate']}Hz")
    print(f"Number of speech segments: {len(result['speech_segments'])}")
    print("Segments (start, end):")
    for seg in result['speech_segments']:
        print(f"  {seg.start:.2f}s - {seg.end:.2f}s")


VAD Results:

File: /content/no_noise.m4a
Total duration: 62.91s
Speech duration: 59.97s
Speech percentage: 95.33%
Sample rate: 16000Hz
Number of speech segments: 5
Segments (start, end):
  0.47s - 15.42s
  15.74s - 30.86s
  31.59s - 40.21s
  40.97s - 46.88s
  46.98s - 62.35s

File: /content/some_noise.m4a
Total duration: 55.30s
Speech duration: 51.05s
Speech percentage: 92.32%
Sample rate: 16000Hz
Number of speech segments: 2
Segments (start, end):
  0.72s - 25.02s
  25.24s - 51.99s

File: /content/heavy_noise.m4a
Total duration: 56.98s
Speech duration: 56.13s
Speech percentage: 98.50%
Sample rate: 16000Hz
Number of speech segments: 1
Segments (start, end):
  0.03s - 56.16s


In [ ]:
# Additional function for explicit timestamp reporting
def print_speech_intervals(results):
    """Prints speech/non-speech intervals with timestamps"""
    for result in results:
        print(f"\nFile: {result['file']}")
        print(f"Total duration: {result['total_duration']:.2f}s")

        # Get all speech segments
        speech_segments = result['speech_segments']

        # Create timeline of speech vs non-speech intervals
        timeline = []
        previous_end = 0.0

        for seg in sorted(speech_segments, key=lambda x: x.start):
            if seg.start > previous_end:
                timeline.append(("non-speech", previous_end, seg.start))
            timeline.append(("SPEECH", seg.start, seg.end))
            previous_end = seg.end

        # Add final non-speech segment if needed
        if previous_end < result['total_duration']:
            timeline.append(("non-speech", previous_end, result['total_duration']))

        # Print formatted timeline
        print("\nInterval Type      Start Time     End Time       Duration")
        print("---------------------------------------------------------")
        for entry in timeline:
            print(f"{entry[0].upper():<12} {entry[1]:>9.2f}s {entry[2]:>9.2f}s {entry[2]-entry[1]:>9.2f}s")

# Add this after your existing results processing
print("\nDetailed Speech/Non-Speech Intervals:")
print_speech_intervals(results)


Detailed Speech/Non-Speech Intervals:

File: /content/no_noise.m4a
Total duration: 62.91s

Interval Type      Start Time     End Time       Duration
---------------------------------------------------------
NON-SPEECH        0.00s      0.47s      0.47s
SPEECH            0.47s     15.42s     14.95s
NON-SPEECH       15.42s     15.74s      0.32s
SPEECH           15.74s     30.86s     15.12s
NON-SPEECH       30.86s     31.59s      0.73s
SPEECH           31.59s     40.21s      8.62s
NON-SPEECH       40.21s     40.97s      0.76s
SPEECH           40.97s     46.88s      5.91s
NON-SPEECH       46.88s     46.98s      0.10s
SPEECH           46.98s     62.35s     15.37s
NON-SPEECH       62.35s     62.91s      0.56s

File: /content/some_noise.m4a
Total duration: 55.30s

Interval Type      Start Time     End Time       Duration
---------------------------------------------------------
NON-SPEECH        0.00s      0.72s      0.72s
SPEECH            0.72s     25.02s     24.30s
NON-SPEECH       25.02s

In [ ]:
import time
import torch
import librosa
import subprocess
from pyannote.audio import Model
from pyannote.audio.pipelines import VoiceActivityDetection

# Initialize model with CUDA if available (reduces latency)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model.from_pretrained("pyannote/segmentation-3.0").to(device)
pipeline = VoiceActivityDetection(segmentation=model).to(device)

pipeline.instantiate({
    "min_duration_on": 0.1,
    "min_duration_off": 0.1
})

def measure_latency(file_path):
    """Measures end-to-end processing latency including conversion"""
    try:
        # Convert M4A to WAV
        start_time = time.time()
        wav_path = file_path.replace('.m4a', '.wav')
        subprocess.run([
            'ffmpeg', '-i', file_path,
            '-ac', '1', '-ar', '16000',
            '-y', wav_path
        ], check=True, capture_output=True)

        # Load audio and keep variables in scope
        waveform, sr = librosa.load(wav_path, sr=16000, mono=True)

        # Process with VAD
        input_tensor = torch.from_numpy(waveform).unsqueeze(0).to(device)
        _ = pipeline({"waveform": input_tensor, "sample_rate": sr})

        return {
            "latency": time.time() - start_time,
            "waveform": waveform,
            "sr": sr
        }

    except Exception as e:
        return {"error": str(e)}

# Test files with proper variable handling
print("Latency Measurement Results:")
for file in files:
    result = measure_latency(file)
    print(f"\n{file.split('/')[-1]}:")

    if "error" in result:
        print(f"  {result['error']}")
    else:
        latency = result["latency"]
        waveform = result["waveform"]
        sr = result["sr"]

        print(f"  Total processing time: {latency:.2f} seconds")
        print(f"  Real-time factor: {(len(waveform)/sr)/latency:.2f}x")

Latency Measurement Results:

no_noise.m4a:
  Total processing time: 0.40 seconds
  Real-time factor: 155.53x

some_noise.m4a:
  Total processing time: 0.30 seconds
  Real-time factor: 185.67x

heavy_noise.m4a:
  Total processing time: 0.31 seconds
  Real-time factor: 186.22x


# CuriousMonkey7/HumAware-VAD

In [ ]:
!pip install -q torch torchaudio librosa pydub ffmpeg-python
!apt-get -qq install ffmpeg

In [ ]:
import torch
import torchaudio
import ffmpeg
import numpy as np
import time
import os
from huggingface_hub import hf_hub_download

In [ ]:
SAMPLE_RATE = 16000
CHUNK_DURATION = 0.032
MIN_CHUNK_SIZE = 512
INITIAL_THRESHOLD = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_metadata(file_path):
    """Extract technical metadata using ffprobe"""
    try:
        probe = ffmpeg.probe(file_path)
        format_info = probe.get('format', {})
        stream_info = next((s for s in probe.get('streams', [])
                          if s.get('codec_type') == 'audio'), None)

        return {
            'format': format_info.get('format_name', 'N/A').split(',')[0],
            'duration': float(format_info.get('duration', 0)),
            'bit_rate': int(format_info.get('bit_rate', 0)),
            'codec': stream_info.get('codec_name', 'N/A') if stream_info else 'N/A',
            'sample_rate': int(stream_info.get('sample_rate', 0)) if stream_info else 0,
            'channels': int(stream_info.get('channels', 1)) if stream_info else 1,
            'file_size': os.path.getsize(file_path)
        }
    except Exception as e:
        print(f"Metadata error: {str(e)}")
        return None

In [ ]:
def convert_to_wav(m4a_path):
    wav_path = m4a_path.replace('.m4a', '.wav')
    try:
        (ffmpeg
         .input(m4a_path)
         .output(wav_path, acodec='pcm_s16le', ac=1, ar=SAMPLE_RATE)
         .overwrite_output()
         .run(capture_stdout=True, capture_stderr=True)
        )
    except ffmpeg.Error as e:
        print(f"FFmpeg error: {e.stderr.decode('utf8')}")
        raise
    return wav_path

In [ ]:
def load_audio(file_path):
    try:
        wav_path = convert_to_wav(file_path)
        if os.path.getsize(wav_path) == 0:
            raise ValueError("Empty audio file after conversion")

        waveform, sr = torchaudio.load(wav_path)

        # Enhanced resampling
        if sr != SAMPLE_RATE:
            waveform = torchaudio.functional.resample(
                waveform,
                orig_freq=sr,
                new_freq=SAMPLE_RATE,
                lowpass_filter_width=64
            )

        # Ensure minimum length
        if waveform.shape[1] < MIN_CHUNK_SIZE:
            waveform = torch.nn.functional.pad(
                waveform,
                (0, MIN_CHUNK_SIZE - waveform.shape[1]),
                mode='reflect'
            )

        return waveform.to(device), SAMPLE_RATE

    except Exception as e:
        print(f"Error loading {file_path}: {str(e)}")
        raise

In [ ]:
model_path = hf_hub_download(
    repo_id="CuriousMonkey7/HumAware-VAD",
    filename="HumAwareVAD.jit",
    revision="main"
)
model = torch.jit.load(model_path, map_location=device).to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


HumAwareVAD.jit:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

RecursiveScriptModule(
  original_name=VADRNNJITMerge
  (_model): RecursiveScriptModule(
    original_name=VADRNNJIT
    (stft): RecursiveScriptModule(
      original_name=STFT
      (padding): RecursiveScriptModule(original_name=ReflectionPad1d)
    )
    (encoder): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=SileroVadBlock
        (se): RecursiveScriptModule(original_name=Identity)
        (activation): RecursiveScriptModule(original_name=ReLU)
        (reparam_conv): RecursiveScriptModule(original_name=Conv1d)
      )
      (1): RecursiveScriptModule(
        original_name=SileroVadBlock
        (se): RecursiveScriptModule(original_name=Identity)
        (activation): RecursiveScriptModule(original_name=ReLU)
        (reparam_conv): RecursiveScriptModule(original_name=Conv1d)
      )
      (2): RecursiveScriptModule(
        original_name=SileroVadBlock
        (se): RecursiveScriptModule(original_name=Identity)
     

In [ ]:
def calculate_adaptive_threshold(probabilities):
    mean = np.mean(probabilities)
    std = np.std(probabilities)
    return max(0.3, min(0.7, mean - 1.5*std))

In [ ]:
def process_audio(file_path):
    try:
        waveform, sr = load_audio(file_path)
        total_samples = waveform.shape[1]
        audio_duration = total_samples / SAMPLE_RATE

        chunk_size = int(SAMPLE_RATE * CHUNK_DURATION)
        num_chunks = int(np.ceil(total_samples / chunk_size))

        # Reflective padding for better edge handling
        padded_samples = num_chunks * chunk_size
        waveform = torch.nn.functional.pad(
            waveform,
            (0, padded_samples - total_samples),
            mode='reflect'
        )

        # Split into validated chunks
        chunks = waveform.unfold(1, chunk_size, chunk_size)
        if chunks.size(1) == 0:
            raise ValueError("No valid chunks created")

        # Process chunks
        speech_probs = []
        total_latency = 0

        model.reset_states()
        for i in range(chunks.size(1)):
            chunk = chunks[:, i].contiguous()

            # Chunk validation
            if chunk.shape[1] < MIN_CHUNK_SIZE:
                raise ValueError(
                    f"Chunk {i} too small: {chunk.shape[1]} samples "
                    f"({chunk.shape[1]/SAMPLE_RATE:.3f}s)"
                )

            chunk = chunk.float() / (torch.max(torch.abs(chunk)) + 1e-7)

            # Process with latency tracking
            start_time = time.time()
            with torch.no_grad():
                prob = torch.sigmoid(model(chunk, sr)).item()
            total_latency += time.time() - start_time

            speech_probs.append(prob)

        # Adaptive thresholding
        threshold = calculate_adaptive_threshold(speech_probs)

        # Generate timestamps
        speech_timestamps = []
        current_speech = None

        for i, prob in enumerate(speech_probs):
            chunk_start = i * CHUNK_DURATION
            chunk_end = (i + 1) * CHUNK_DURATION

            if prob > threshold:
                if current_speech:
                    current_speech[1] = chunk_end
                else:
                    current_speech = [chunk_start, chunk_end]
            else:
                if current_speech:
                    speech_timestamps.append(tuple(current_speech))
                    current_speech = None

        if current_speech:
            speech_timestamps.append(tuple(current_speech))

        # Calculate non-speech intervals
        non_speech = []
        prev_end = 0.0

        for start, end in speech_timestamps:
            if start > prev_end:
                non_speech.append((prev_end, start))
            prev_end = end

        if prev_end < audio_duration:
            non_speech.append((prev_end, audio_duration))

        return {
            "speech": speech_timestamps,
            "non_speech": non_speech,
            "total_duration": audio_duration,
            "latency": total_latency,
            "threshold": threshold,
            "chunk_size": CHUNK_DURATION,
            "probs": speech_probs
        }

    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

In [ ]:
def print_results(result, filename):
    if not result:
        return

    print(f"\n{'='*40}")
    print(f" Analysis Report: {filename}")
    print(f"{'='*40}")

    if result.get('metadata'):
        meta = result['metadata']
        print("\n[File Metadata]")
        print(f" - Format: {meta['format']}")
        print(f" - Duration: {meta['duration']:.2f}s (original)")
        print(f" - Codec: {meta['codec'].upper()}")
        print(f" - Sample Rate: {meta['sample_rate']} Hz (original)")
        print(f" - Channels: {meta['channels']}")
        print(f" - Bit Rate: {meta['bit_rate']//1000} kbps")
        print(f" - File Size: {meta['file_size']//1024} KB")

    print(f"\n Total Duration: {result['total_duration']:.2f}s (processed)")
    print(f" Processing Latency: {result['latency']:.4f}s")
    print(f" Adaptive Threshold: {result['threshold']:.2f}")

    print("\n Speech Intervals:")
    if result['speech']:
        for i, (start, end) in enumerate(result['speech'], 1):
            print(f"  {i:02d}. {start:7.3f}s - {end:7.3f}s | Duration: {end-start:.3f}s")
    else:
        print(" No speech detected")

    print("\n Non-Speech Intervals:")
    if result['non_speech']:
        for i, (start, end) in enumerate(result['non_speech'], 1):
            print(f"  {i:02d}. {start:7.3f}s - {end:7.3f}s | Duration: {end-start:.3f}s")
    else:
        print(" Continuous speech detected")

    total_speech = sum(end-start for start, end in result['speech'])
    speech_percent = (total_speech / result['total_duration']) * 100
    noise_level = 1 - (total_speech / result['total_duration'])

    print(f"\nStatistics:")
    print(f"- Speech Percentage: {speech_percent:6.1f}%")
    print(f"- Noise Level:       {noise_level:6.1%}")
    print(f"- Chunks Processed:  {len(result['probs'])}")
    print(f"- Avg Chunk Time:    {(result['latency']/len(result['probs']))*1000:.2f}ms")


files = [
    "/content/no_noise.m4a",
    "/content/some_noise.m4a",
    "/content/heavy_noise.m4a"
]

for file in files:
    result = process_audio(file)
    if result:
        print_results(result, os.path.basename(file))
    print("\n" + "="*40 + "\n")


 Analysis Report: no_noise.m4a

 Total Duration: 62.91s (processed)
 Processing Latency: 2.2503s
 Adaptive Threshold: 0.70

 Speech Intervals:
  01.   0.608s -   1.280s | Duration: 0.672s
  02.   1.312s -   1.344s | Duration: 0.032s
  03.   1.376s -  11.776s | Duration: 10.400s
  04.  11.808s -  12.288s | Duration: 0.480s
  05.  12.320s -  15.808s | Duration: 3.488s
  06.  15.840s -  17.696s | Duration: 1.856s
  07.  17.728s -  19.616s | Duration: 1.888s
  08.  19.680s -  19.776s | Duration: 0.096s
  09.  19.936s -  19.968s | Duration: 0.032s
  10.  20.032s -  26.976s | Duration: 6.944s
  11.  27.008s -  31.648s | Duration: 4.640s
  12.  31.680s -  46.304s | Duration: 14.624s
  13.  46.336s -  51.232s | Duration: 4.896s
  14.  51.264s -  57.984s | Duration: 6.720s
  15.  58.016s -  62.624s | Duration: 4.608s
  16.  62.656s -  62.912s | Duration: 0.256s

 Non-Speech Intervals:
  01.   0.000s -   0.608s | Duration: 0.608s
  02.   1.280s -   1.312s | Duration: 0.032s
  03.   1.344s -   1

# Output Modification Analysis
- add threshold mode parameter
- chunk processing
- adding noise compensation

Interpretation Mapping:
- Speech segments:	Temporal boundaries of voice activity
- Probability array:	Model confidence per time quantum
- Threshold value:	System's detection sensitivity
- Latency metrics:	Computational efficiency

# Silero VAD

In [ ]:
!apt-get install -y ffmpeg
!pip install -q torch torchaudio pydub

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 M

In [ ]:
import torch
from pydub import AudioSegment
import os
import time

In [ ]:
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=False)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


In [ ]:
def convert_to_wav(input_path):
    audio = AudioSegment.from_file(input_path)
    wav_path = os.path.splitext(input_path)[0] + ".wav"
    audio.set_frame_rate(16000).set_channels(1).export(wav_path, format="wav")
    return wav_path

In [ ]:
def process_audio(file_path):
    wav_path = convert_to_wav(file_path)
    waveform = read_audio(wav_path, sampling_rate=16000)

    start_time = time.time()
    speech_timestamps = get_speech_timestamps(waveform, model, sampling_rate=16000)
    latency = time.time() - start_time

    total_duration = len(waveform) / 16000
    speech_duration = sum([(ts['end']-ts['start'])/16000 for ts in speech_timestamps])

    return {
        "timestamps": speech_timestamps,
        "latency": round(latency, 4),
        "accuracy": round(speech_duration/total_duration, 4),
        "metadata": {
            "sample_rate": 16000,
            "total_duration": round(total_duration, 2),
            "noise_level": os.path.basename(file_path).split('_')[0]
        }
    }

In [ ]:
results = {}
for file in ['/content/no_noise.m4a', '/content/some_noise.m4a', '/content/heavy_noise.m4a']:
    if os.path.exists(file):
        results[file] = process_audio(file)

for file, data in results.items():
    print(f"\n## Results for {os.path.basename(file)}")
    print(f"\nMetadata:")
    print(f"- Noise Level: {data['metadata']['noise_level']}")
    print(f"- Total Duration: {data['metadata']['total_duration']}s")
    print(f"- Speech Detected: {len(data['timestamps'])} segments")
    print(f"- Processing Latency: {data['latency']}s")
    print(f"- Estimated Accuracy: {data['accuracy']*100}%")
    print("\nTimestamps (seconds):")
    for ts in data['timestamps']:
        print(f"  {ts['start']/16000:.2f}s - {ts['end']/16000:.2f}s")


## Results for no_noise.m4a

Metadata:
- Noise Level: no
- Total Duration: 62.91s
- Speech Detected: 7 segments
- Processing Latency: 2.574s
- Estimated Accuracy: 96.04%

Timestamps (seconds):
  0.48s - 15.71s
  15.81s - 31.20s
  31.62s - 34.88s
  35.04s - 40.48s
  40.99s - 52.86s
  52.99s - 56.96s
  57.12s - 62.40s

## Results for some_noise.m4a

Metadata:
- Noise Level: some
- Total Duration: 55.3s
- Speech Detected: 9 segments
- Processing Latency: 2.2359s
- Estimated Accuracy: 89.52%

Timestamps (seconds):
  0.77s - 13.82s
  13.95s - 16.48s
  16.64s - 25.12s
  25.38s - 32.57s
  32.77s - 35.84s
  36.26s - 39.29s
  39.49s - 47.01s
  47.23s - 49.69s
  49.92s - 52.09s

## Results for heavy_noise.m4a

Metadata:
- Noise Level: heavy
- Total Duration: 56.98s
- Speech Detected: 14 segments
- Processing Latency: 1.8647s
- Estimated Accuracy: 86.95%

Timestamps (seconds):
  0.83s - 7.04s
  7.43s - 10.30s
  10.40s - 12.29s
  12.67s - 19.10s
  19.33s - 23.36s
  23.65s - 29.85s
  31.11s - 34.2

# Modification Analysis
- Speech Probability Threshold
- Negative Threshold
- Time-Based Parameters: Minimum Speech Duration and Maximum Speech Duration
- Padding
- Sample Rate

# Custom VAD

In [ ]:
!pip install librosa pydub matplotlib scikit-learn ipython pandas -q
!apt install ffmpeg -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.3 MB/s eta 0:00:00
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
import time
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from pydub import AudioSegment
from IPython.display import Audio, display
import pandas as pd
from google.colab import files

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.generic):
            return obj.item()
        return super().default(obj)

In [ ]:
class AudioConverter:
    @staticmethod
    def convert_with_timing(input_path, output_path):
        start_time = time.time()
        try:
            audio = AudioSegment.from_file(input_path)
            audio = audio.set_channels(1)  # Convert to mono
            audio.export(output_path, format="wav")
            return time.time() - start_time, None
        except Exception as e:
            return 0, str(e)


audio_files = ["/content/no_noise.m4a", "/content/some_noise.m4a", "/content/heavy_noise.m4a"]
wav_files = []
conversion_metrics = {}

for file in audio_files:
    if os.path.exists(file):
        base_name = os.path.basename(file).split('.')[0]
        wav_path = f"/content/{base_name}.wav"
        conv_time, error = AudioConverter.convert_with_timing(file, wav_path)
        if error:
            print(f"Conversion failed for {file}: {error}")
            continue
        conversion_metrics[base_name] = {
            "conversion_time": conv_time,
            "original_format": file.split('.')[-1]
        }
        wav_files.append(wav_path)

In [ ]:
class EnhancedVAD:
    def __init__(self, frame_length=1024, hop_length=512, energy_percentile=85):
        self.frame_length = frame_length
        self.hop_length = hop_length
        self.energy_percentile = energy_percentile
        self.latency = 0.0

    def _get_speech_intervals(self, speech_frames, sr):
        frame_duration = self.hop_length / sr
        intervals = []
        current_start = None

        for i, is_speech in enumerate(speech_frames):
            frame_start = i * frame_duration
            if is_speech:
                current_start = current_start or frame_start
            else:
                if current_start is not None:
                    intervals.append((round(current_start, 3), round(frame_start, 3)))
                    current_start = None

        if current_start is not None:
            intervals.append((round(current_start, 3), round(frame_start + frame_duration, 3)))

        return intervals

    def process(self, audio_path):
        start_time = time.time()
        try:
            y, sr = librosa.load(audio_path, sr=None)
            frames = librosa.util.frame(y, frame_length=self.frame_length,
                                      hop_length=self.hop_length)

            energy = np.sqrt(np.mean(frames**2, axis=0))
            threshold = np.percentile(energy, self.energy_percentile)
            speech_frames = energy > threshold

            intervals = self._get_speech_intervals(speech_frames, sr)

            self.latency = time.time() - start_time
            return {
                "intervals": intervals,
                "threshold": float(threshold),  # Convert to native float
                "sr": sr,
                "error": None
            }
        except Exception as e:
            return {"error": str(e)}

In [ ]:
def format_intervals(intervals):
    return "\n".join([f"{start:.3f}-{end:.3f}s" for start, end in intervals])

report_df = pd.DataFrame(results)
report_df["speech_segments"] = report_df["speech_intervals"].apply(format_intervals)

print("\n=== Final Detection Report ===")
display(report_df[["file", "total_duration", "speech_duration",
                 "accuracy", "latency", "speech_segments"]])

metadata = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "audio_files": {
        "original": [os.path.basename(f) for f in audio_files],
        "converted": [os.path.basename(f) for f in wav_files]
    },
    "conversion_metrics": conversion_metrics,
    "processing_metrics": processing_metrics,
    "parameters": {
        "frame_length": 1024,
        "hop_length": 512,
        "energy_percentile": 85
    }
}

with open("/content/vad_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2, cls=NumpyEncoder)

print("\nMetadata saved to: /content/vad_metadata.json")

# Audio playback function
def play_with_markers(file_index=0):
    audio_path = wav_files[file_index]
    y, sr = librosa.load(audio_path)
    display(Audio(y, rate=sr))
    print(f"Speech segments for {os.path.basename(audio_path)}:")
    print(report_df.iloc[file_index]["speech_segments"])


=== Final Detection Report ===


,file,total_duration,speech_duration,accuracy,latency,speech_segments
0,no_noise.wav,62.912,9.443,15.0,0.021,0.565-0.576s\n0.661-0.672s\n1.984-1.995s\n2.25...
1,some_noise.wav,55.296,8.303,15.0,0.016,0.832-0.864s\n0.928-1.088s\n1.163-1.269s\n1.36...
2,heavy_noise.wav,56.981,8.542,15.0,0.017,0.235-0.256s\n0.341-0.352s\n0.395-0.416s\n0.52...



Metadata saved to: /content/vad_metadata.json


# webrtc

In [ ]:
!pip install webrtcvad pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp311-cp311-linux_x86_64.whl size=73499 sha256=1e282def9b86bd99a294a4e0d892bce1951404d438eb379d8570a4ebc2346f8a
  Stored in directory: /root/.cache/pip/wheels/94/65/3f/292d0b656be33d1c801831201c74b5f68f41a2ae465ff2ee2f
Successfully built webrtcvad


In [ ]:
import webrtcvad
from pydub import AudioSegment
import numpy as np
import sys
import time

In [ ]:
import polars as pl

pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_width_chars(200)
pl.Config.set_ascii_tables()

polars.config.Config

In [ ]:
np.set_printoptions(threshold=sys.maxsize, linewidth=200)
sys.setrecursionlimit(10000)

In [ ]:
def analyze_audio(file_path, vad_mode=3, frame_duration=30):
    """Analyze audio with WebRTC VAD and return detailed metrics"""
    start_time = time.time()

    vad = webrtcvad.Vad(vad_mode)
    audio = AudioSegment.from_file(file_path)

    # Metadata collection
    metadata = {
        'original_duration': len(audio) / 1000,
        'sample_rate': audio.frame_rate,
        'channels': audio.channels,
        'vad_mode': vad_mode,
        'frame_duration': frame_duration
    }

    # Audio preprocessing
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    samples = np.array(audio.get_array_of_samples(), dtype=np.int16)

    frame_size = int(16000 * frame_duration / 1000)
    results = []
    speech_segments = []
    current_segment = None
    total_speech = 0

    for i in range(0, len(samples), frame_size):
        frame = samples[i:i+frame_size]
        if len(frame) < frame_size:
            frame = np.pad(frame, (0, frame_size - len(frame)), 'constant')

        timestamp = i / 16000
        is_speech = vad.is_speech(frame.tobytes(), 16000)

        results.append({
            'start': timestamp,
            'end': timestamp + (frame_duration/1000),
            'speech': is_speech
        })

        if is_speech:
            if not current_segment:
                current_segment = {'start': timestamp, 'end': None}
            total_speech += frame_duration/1000
        else:
            if current_segment:
                current_segment['end'] = timestamp
                speech_segments.append(current_segment)
                current_segment = None

    # Handle final segment
    if current_segment:
        current_segment['end'] = len(samples)/16000
        speech_segments.append(current_segment)

    latency = time.time() - start_time
    return {
        'metadata': metadata,
        'frame_results': results,
        'statistics': {
            'total_speech': total_speech,
            'speech_percentage': (total_speech / metadata['original_duration']) * 100,
            'speech_segments': speech_segments,
            'latency': latency,
            'frame_count': len(results)
        }
    }

# Analysis with full output
noise_levels = {
    'no_noise': '/content/no_noise.m4a',
    'moderate_noise': '/content/some_noise.m4a',
    'high_noise': '/content/heavy_noise.m4a'
}

for scenario, file_path in noise_levels.items():
    print(f"\n{scenario.replace('_', ' ').upper()} ANALYSIS")
    for mode in [1, 3]:
        result = analyze_audio(file_path, vad_mode=mode)
        stats = result['statistics']

         # Print metadata
        print(f"\nVAD Mode {mode} Metadata:")
        for key, value in metadata.items():
            print(f"{key.capitalize().replace('_', ' ')}: {value}")

        # Print results in specified format
        print(f"\nVAD Mode {mode} Results:")
        print("Timestamps (seconds):")
        for seg in stats['speech_segments']:
            print(f"  {seg['start']:.2f}s - {seg['end']:.2f}s")

        # Print performance metrics
        print(f"\nProcessing latency: {stats['latency']:.4f}s")
        print(f"Total speech duration: {stats['total_speech']:.2f}s")
        print(f"Speech percentage: {stats['speech_percentage']:.1f}%")


NO NOISE ANALYSIS

VAD Mode 1 Metadata:
Timestamp: 2025-04-01 09:03:52
Audio files: {'original': ['no_noise.m4a', 'some_noise.m4a', 'heavy_noise.m4a'], 'converted': ['no_noise.wav', 'some_noise.wav', 'heavy_noise.wav']}
Conversion metrics: {'no_noise': {'conversion_time': 0.33206725120544434, 'original_format': 'm4a'}, 'some_noise': {'conversion_time': 0.2904033660888672, 'original_format': 'm4a'}, 'heavy_noise': {'conversion_time': 0.2911033630371094, 'original_format': 'm4a'}}
Processing metrics: {'no_noise.wav': {'latency': 0.02091670036315918, 'threshold': 0.016745520755648613}, 'some_noise.wav': {'latency': 0.016484975814819336, 'threshold': 0.019618181511759758}, 'heavy_noise.wav': {'latency': 0.016586780548095703, 'threshold': 0.0725068598985672}}
Parameters: {'frame_length': 1024, 'hop_length': 512, 'energy_percentile': 85}

VAD Mode 1 Results:
Timestamps (seconds):
  0.03s - 0.21s
  0.48s - 8.22s
  8.40s - 13.05s
  13.08s - 15.75s
  15.84s - 16.59s
  16.77s - 23.88s
  23.94s 

# Modification Analysis
- aggresiveness mode
- changing sample rate, channels and bit depth